# GRU Classification with MR Dataset
<hr>

We will build a text classification model using GRU model on the MR Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [39]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [40]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [41]:
corpus = pd.read_pickle('../../../0_data/MR/MR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10662, 3)


sentence  label  split
0                       simplistic , silly and tedious .      0  train
1      it 's so laddish and juvenile , only teenage b...      0  train
2      exploitative and largely devoid of the depth o...      0  train
3      garbus discards the potential for pathological...      0  train
4      a visually flashy but narratively opaque and e...      0  train
...                                                  ...    ...    ...
10657  both exuberantly romantic and serenely melanch...      1  train
10658  mazel tov to a film about a family 's joyous l...      1  train
10659  standing in the shadows of motown is the best ...      1  train
10660  it 's nice to see piscopo again after all thes...      1  train
10661  provides a porthole into that noble , tremblin...      1  train

[10662 rows x 3 columns]

In [42]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10662 entries, 0 to 10661
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10662 non-null  object
 1   label     10662 non-null  int32 
 2   split     10662 non-null  object
dtypes: int32(1), object(2)
memory usage: 208.4+ KB


In [43]:
corpus.groupby( by='label').count()

sentence  split
label                 
0          5331   5331
1          5331   5331

In [44]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [45]:
sentences[0]

'simplistic , silly and tedious .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [46]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [47]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  a visually flashy but narratively opaque and emotionally vapid exercise in style and mystification .
Into a sequence of int: [3, 544, 1838, 13, 3909, 3366, 4, 658, 2629, 416, 10, 236, 4, 10112]
Into a padded sequence: [    3   544  1838    13  3909  3366     4   658  2629   416    10   236
     4 10112     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


In [48]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
is 7
's 8
it 9
in 10
18760


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## GRU Model

In [49]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [50]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_41 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_41 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_41 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 440,673
Non-trainable params: 0
_________________________________________________________________


In [51]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [52]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/15
300/300 [==============================] - 122s 372ms/step - loss: 0.6322 - accuracy: 0.6182 - val_loss: 0.4879 - val_accuracy: 0.7648
Epoch 2/15
300/300 [==============================] - 119s 397ms/step - loss: 0.2375 - accuracy: 0.9085 - val_loss: 0.5737 - val_accuracy: 0.7807
Epoch 3/15
300/300 [==============================] - 120s 401ms/step - loss: 0.0724 - accuracy: 0.9762 - val_loss: 0.7641 - val_accuracy: 0.7582
Epoch 4/15
300/300 [==============================] - 117s 390ms/step - loss: 0.0298 - accuracy: 0.9917 - val_loss: 1.0340 - val_accuracy: 0.7685
Epoch 5/15
300/300 [==============================] - 112s 374ms/step - loss: 0.0109 - accuracy: 0.9966 - val_loss: 1.1488 - val_accuracy: 0.7657
Epoch 6/15
300/300 [==============================] - 113s 377ms/step - loss: 0.0066 - accuracy: 0.9987 - val_loss: 1.3445 - val_accuracy: 0.7723
Epoch 7/15
300/300 [==============================] - 113s 377ms/step - loss: 0.0027 - accuracy: 0.9995 - val_l

Epoch 4/15
300/300 [==============================] - 121s 404ms/step - loss: 0.0220 - accuracy: 0.9952 - val_loss: 0.9008 - val_accuracy: 0.7570
Epoch 5/15
300/300 [==============================] - 121s 403ms/step - loss: 0.0126 - accuracy: 0.9959 - val_loss: 1.0815 - val_accuracy: 0.7561
Epoch 6/15
300/300 [==============================] - 121s 405ms/step - loss: 0.0046 - accuracy: 0.9988 - val_loss: 1.1452 - val_accuracy: 0.7608
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 76.82926654815674
Training 9: 
Epoch 1/15
300/300 [==============================] - 123s 374ms/step - loss: 0.6291 - accuracy: 0.6133 - val_loss: 0.4601 - val_accuracy: 0.7824
Epoch 2/15
300/300 [==============================] - 107s 355ms/step - loss: 0.2401 - accuracy: 0.9024 - val_loss: 0.5134 - val_accuracy: 0.7777
Epoch 3/15
300/300 [==============================] - 105s 349ms/step - loss: 0.0696 - accuracy: 0.9775 - val_loss: 0.6872 - val_accuracy: 0

## Summary

In [53]:
record

acc1       acc2       acc3       acc4      acc5       acc6       acc7  \
0  78.069353  77.788192  77.861166  78.986865  75.51595  76.454031  77.861166   

        acc8       acc9      acc10        AVG  
0  76.829267  78.236395  74.296433  77.189882

In [54]:
report = record
report = report.to_excel('GRU_MR.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [55]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [56]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [57]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [58]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 16448 words present from 18760 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [59]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    
    embed_matrix = np.zeros((vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.word_vec(word)
            
        # initialize the unknown word with standard normal distribution values
        else:
            embed_matrix[idx] = np.random.randn(emb_dim)
            
    return embed_matrix

In [60]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i)
em_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.4071945 ,  0.67546439, -0.54825804, ..., -0.37580828,
        -1.2612194 ,  1.1520491 ],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## GRU Model

In [61]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [62]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_52 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_52 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_52 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 140,673
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [63]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [64]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/15
300/300 [==============================] - 101s 294ms/step - loss: 0.6393 - accuracy: 0.6159 - val_loss: 0.5223 - val_accuracy: 0.7395
Epoch 2/15
300/300 [==============================] - 89s 297ms/step - loss: 0.4563 - accuracy: 0.7799 - val_loss: 0.5240 - val_accuracy: 0.7320
Epoch 3/15
300/300 [==============================] - 88s 294ms/step - loss: 0.4209 - accuracy: 0.8046 - val_loss: 0.5416 - val_accuracy: 0.7216
Epoch 4/15
300/300 [==============================] - 88s 295ms/step - loss: 0.3873 - accuracy: 0.8197 - val_loss: 0.5281 - val_accuracy: 0.7366
Epoch 5/15
300/300 [==============================] - 88s 292ms/step - loss: 0.3540 - accuracy: 0.8408 - val_loss: 0.5686 - val_accuracy: 0.7273
Epoch 6/15
300/300 [==============================] - 88s 294ms/step - loss: 0.3185 - accuracy: 0.8654 - val_loss: 0.5939 - val_accuracy: 0.7366
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 73.94564151763916

Epoch 11/15
300/300 [==============================] - 43s 144ms/step - loss: 0.1594 - accuracy: 0.9350 - val_loss: 0.6936 - val_accuracy: 0.7645
Epoch 12/15
300/300 [==============================] - 44s 147ms/step - loss: 0.1405 - accuracy: 0.9433 - val_loss: 0.7612 - val_accuracy: 0.7617
Epoch 13/15
300/300 [==============================] - 43s 144ms/step - loss: 0.1069 - accuracy: 0.9588 - val_loss: 0.7575 - val_accuracy: 0.7523
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 77.39211916923523
Training 6: 
Epoch 1/15
300/300 [==============================] - 53s 135ms/step - loss: 0.6454 - accuracy: 0.6023 - val_loss: 0.5155 - val_accuracy: 0.7280
Epoch 2/15
300/300 [==============================] - 28s 93ms/step - loss: 0.4556 - accuracy: 0.7845 - val_loss: 0.4937 - val_accuracy: 0.7364
Epoch 3/15
300/300 [==============================] - 28s 94ms/step - loss: 0.4143 - accuracy: 0.8080 - val_loss: 0.4992 - val_accuracy: 0.7373

Epoch 10/15
300/300 [==============================] - 52s 173ms/step - loss: 0.1903 - accuracy: 0.9265 - val_loss: 0.5887 - val_accuracy: 0.7833
Epoch 11/15
300/300 [==============================] - 52s 175ms/step - loss: 0.1579 - accuracy: 0.9398 - val_loss: 0.6731 - val_accuracy: 0.7758
Epoch 12/15
300/300 [==============================] - 51s 170ms/step - loss: 0.1260 - accuracy: 0.9541 - val_loss: 0.7472 - val_accuracy: 0.7692
Epoch 13/15
300/300 [==============================] - 51s 170ms/step - loss: 0.0877 - accuracy: 0.9674 - val_loss: 0.8559 - val_accuracy: 0.7674
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping
Test Accuracy: 78.70544195175171

        acc1      acc2      acc3       acc4       acc5       acc6       acc7  \
0  73.945642  73.66448  74.76548  77.110696  77.392119  75.422138  77.579737   

        acc8       acc9      acc10        AVG  
0  70.544088  75.703567  78.705442  75.483339  



## Summary

In [65]:
record2

acc1      acc2      acc3       acc4       acc5       acc6       acc7  \
0  73.945642  73.66448  74.76548  77.110696  77.392119  75.422138  77.579737   

        acc8       acc9      acc10        AVG  
0  70.544088  75.703567  78.705442  75.483339

In [66]:
report = record2
report = report.to_excel('GRU_MR_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## GRU Model

In [67]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.GRU(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [68]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_63"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_63 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_63 (Bidirectio (None, 128)               140544    
_________________________________________________________________
dropout_63 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 129       
Total params: 440,673
Trainable params: 440,673
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [69]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [70]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/15
300/300 [==============================] - 170s 526ms/step - loss: 0.6325 - accuracy: 0.6283 - val_loss: 0.4549 - val_accuracy: 0.7844
Epoch 2/15
300/300 [==============================] - 155s 515ms/step - loss: 0.2966 - accuracy: 0.8803 - val_loss: 0.4419 - val_accuracy: 0.7938
Epoch 3/15
300/300 [==============================] - 155s 515ms/step - loss: 0.1250 - accuracy: 0.9580 - val_loss: 0.5194 - val_accuracy: 0.7873
Epoch 4/15
300/300 [==============================] - 154s 514ms/step - loss: 0.0483 - accuracy: 0.9859 - val_loss: 0.7900 - val_accuracy: 0.7779
Epoch 5/15
300/300 [==============================] - 157s 522ms/step - loss: 0.0246 - accuracy: 0.9928 - val_loss: 0.7976 - val_accuracy: 0.7760
Epoch 6/15
300/300 [==============================] - 158s 525ms/step - loss: 0.0085 - accuracy: 0.9980 - val_loss: 0.9283 - val_accuracy: 0.7751
Epoch 7/15
300/300 [==============================] - 157s 525ms/step - loss: 0.0047 - accuracy: 0.9986 - val_l

300/300 [==============================] - 131s 438ms/step - loss: 0.1174 - accuracy: 0.9593 - val_loss: 1.0706 - val_accuracy: 0.7092
Epoch 4/15
300/300 [==============================] - 126s 421ms/step - loss: 0.0447 - accuracy: 0.9881 - val_loss: 1.4775 - val_accuracy: 0.6951
Epoch 5/15
300/300 [==============================] - 126s 420ms/step - loss: 0.0155 - accuracy: 0.9963 - val_loss: 1.7903 - val_accuracy: 0.6961
Epoch 6/15
300/300 [==============================] - 125s 417ms/step - loss: 0.0088 - accuracy: 0.9984 - val_loss: 2.1346 - val_accuracy: 0.6970
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 72.04502820968628
Training 8: 
Epoch 1/15
300/300 [==============================] - 110s 324ms/step - loss: 0.6366 - accuracy: 0.6200 - val_loss: 0.4590 - val_accuracy: 0.7833
Epoch 2/15
300/300 [==============================] - 93s 309ms/step - loss: 0.2969 - accuracy: 0.8757 - val_loss: 0.5266 - val_accuracy: 0.7514
Epoch 

## Summary

In [71]:
record3

acc1       acc2       acc3       acc4      acc5       acc6       acc7  \
0  79.381442  77.600747  76.735461  78.517824  78.61163  76.641649  72.045028   

        acc8       acc9      acc10        AVG  
0  78.330207  77.392119  76.923078  77.217919

In [72]:
report = record3
report = report.to_excel('GRU_MR_3.xlsx', sheet_name='dynamic')